# Ensemble with previous submissions

Voting classifier with previous predictions, to improve robustness

In [29]:
import pandas as pd
import os
import datetime

In [30]:
dfs = []
for file in os.listdir("submissions"):
    df = pd.read_csv(f"submissions/{file}")
    # Rename "category" column to include the file name (without extension)
    name = os.path.splitext(file)[0]   # e.g. "submission1.csv" → "submission1"
    df = df.rename(columns={"category": f"category_{name}"})
    dfs.append(df)

In [31]:
# Merge them all on "id" using successive .merge
ensemble = dfs[0]
for df in dfs[1:]:
    ensemble = ensemble.merge(df, on="id")

ensemble.head()

,id,category_Transfer_main_09_30_19_41,category_BigClassifier_best_09_29_11_13,category_Transfer_main_10_01_13_24
0,0.jpeg,scoiattolo,scoiattolo,scoiattolo
1,1.jpeg,cavallo,cavallo,cavallo
2,10.jpeg,scoiattolo,scoiattolo,scoiattolo
3,100.jpeg,pecora,pecora,pecora
4,1000.jpeg,cavallo,cavallo,cavallo


In [32]:
category_cols = [col for col in ensemble.columns if col.startswith("category_")]

def choose_best(row):
    # Count frequencies of the categories in this row
    counts = row[category_cols].value_counts()
    if counts.iloc[0] > 1:
        # There is a clear majority
        return counts.index[0]
    else:
        # All three are different → fall back to the special column
        return row["category_Transfer_main_10_01_13_24"]

ensemble["category"] = ensemble.apply(choose_best, axis=1)
ensemble.set_index("id", inplace= True)

ensemble.head()

,category_Transfer_main_09_30_19_41,category_BigClassifier_best_09_29_11_13,category_Transfer_main_10_01_13_24,category
id,,,,
0.jpeg,scoiattolo,scoiattolo,scoiattolo,scoiattolo
1.jpeg,cavallo,cavallo,cavallo,cavallo
10.jpeg,scoiattolo,scoiattolo,scoiattolo,scoiattolo
100.jpeg,pecora,pecora,pecora,pecora
1000.jpeg,cavallo,cavallo,cavallo,cavallo


In [34]:
timestamp = datetime.datetime.now().strftime("%m_%d_%H_%M")
ensemble["category"].to_csv(f"submissions/Ensemble_{timestamp}.csv")